In [199]:
# Standard Libraries
import os
import pydicom
from PIL import Image
import yaml

# Data Manipulation Libraries
import pandas as pd
import numpy as np


# Machine Learning Libraries
import torch
from sklearn.model_selection import train_test_split

# BYOL library
from byol_pytorch import BYOL

**Define Parameters**

In [200]:
# File path
EXTRACTED_PATH = '/cluster/home/bjorneme/projects/Data/vinbigdata-chest-xray-abnormalities-detection-extracted'
train_csv = os.path.join(EXTRACTED_PATH, 'train.csv')

# Disease labels
disease_labels = [
    "Aortic enlargement", "Atelectasis", "Calcification", "Cardiomegaly",
    "Consolidation", "ILD", "Infiltration", "Lung Opacity",
    "Nodule/Mass", "Other lesion", "Pleural effusion", "Pleural thickening",
    "Pneumothorax", "Pulmonary fibrosis"
]

# Device Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [201]:
def open_dicom_image(path):
    ds = pydicom.dcmread(path)
    img = ds.pixel_array.astype(np.float32)
    img = ((img - img.min()) / (img.max() - img.min()) * 255).astype(np.uint8)
    return Image.fromarray(img)

In [202]:
def convert_annotation_to_yolo(group, img_size_override=None):
    # Size image
    img_path = group.iloc[0]['Path']
    img_width, img_height = open_dicom_image(img_path).size
    
    lines = []
    for _, row in group.iterrows():
        x_min, y_min, x_max, y_max = row[['x_min','y_min','x_max','y_max']]
        x_center = ((x_min + x_max) / 2) / img_width
        y_center = ((y_min + y_max) / 2) / img_height
        box_width = (x_max - x_min) / img_width
        box_height = (y_max - y_min) / img_height
        lines.append(f"{int(row['class_id'])} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}")
    return img_path, lines

In [203]:
def load_labels(csv_path, image_path, disease_labels):
    df = pd.read_csv(csv_path)
    for d in disease_labels:
        df[d] = df['class_name'].str.contains(d).astype(int)
    df['No finding'] = df['class_name'].str.contains('No finding').astype(int)
    df['Path'] = df['image_id'].apply(lambda x: os.path.join(image_path, 'train', f"{x}.dicom"))
    return df

In [204]:
def process_split(df, split_name, base_dir):
    for image_id, group in df.groupby('image_id'):
        img_path, yolo_lines = convert_annotation_to_yolo(group)
        img_filename = os.path.basename(img_path)
        ext = os.path.splitext(img_filename)[1].lower()
        img_dir = os.path.join(base_dir, "images", split_name)
        label_dir = os.path.join(base_dir, "labels", split_name)
        os.makedirs(img_dir, exist_ok=True)
        os.makedirs(label_dir, exist_ok=True)
        
        pil_img = open_dicom_image(img_path)
        img_filename = os.path.splitext(img_filename)[0] + ".png"
        pil_img.save(os.path.join(img_dir, img_filename))
        
        label_filename = os.path.splitext(img_filename)[0] + ".txt"
        with open(os.path.join(label_dir, label_filename), "w") as f:
            f.write("\n".join(yolo_lines))

In [205]:
df = load_labels(train_csv, EXTRACTED_PATH, disease_labels)
df = df[df['class_id'] != 14]
print(f"Train size: {df.shape[0]}")

unique_ids = df['image_id'].unique()
train_ids, val_ids = train_test_split(unique_ids, test_size=0.2, random_state=42)
train_df, val_df = df[df['image_id'].isin(train_ids)], df[df['image_id'].isin(val_ids)]
print(f"Train images: {len(train_ids)}, Val images: {len(val_ids)}")

for split in ['train', 'val']:
    os.makedirs(os.path.join(EXTRACTED_PATH, "images", split), exist_ok=True)
    os.makedirs(os.path.join(EXTRACTED_PATH, "labels", split), exist_ok=True)

print("Processing training set...")
# process_split(train_df, "train", EXTRACTED_PATH)
print("Processing validation set...")
# process_split(val_df, "val", EXTRACTED_PATH)

Train size: 36096
Train images: 3515, Val images: 879
Processing training set...
Processing validation set...


In [206]:
data_yaml = {
    "train": os.path.join(os.getcwd(), EXTRACTED_PATH, "images", "train"),
    "val": os.path.join(os.getcwd(), EXTRACTED_PATH, "images", "val"),
    "nc": 14,
    "names": disease_labels
}

with open("data.yaml", "w") as f:
    yaml.dump(data_yaml, f, default_flow_style=False)

In [208]:
from ultralytics import YOLO

model = YOLO("yolo11n.pt")
model.to(device)

100%|██████████| 5.35M/5.35M [00:00<00:00, 45.1MB/s]


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_

In [210]:
# Train with image size 224 (instead of 640)
results = model.train(data="data.yaml", epochs=100, imgsz=224)

New https://pypi.org/project/ultralytics/8.3.89 available 😃 Update with 'pip install -U ultralytics'
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show

  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  8                  -1  1    346112  ultralytics.nn.modules.block.C3k2            [256, 256, 1, True]           
  9                  -1  1    164608  ultralytics.nn.modules.block.SPPF            [256, 256, 5]                 
 10                  -1  1    249728  ultralytics.nn.modules.block.C2PSA           [256, 256, 1]                 
 11                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None

train: Scanning /cluster/home/bjorneme/projects/Data/vinbigdata-chest-xray-abnormalities-detection-extracted/labels/train.cache... 3515 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3515/3515 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /cluster/home/bjorneme/projects/Data/vinbigdata-chest-xray-abnormalities-detection-extracted/labels/val.cache... 879 images, 0 backgrounds, 0 corrupt: 100%|██████████| 879/879 [00:00<?, ?it/s]


...

                   all        879       7139      0.327       0.26      0.231      0.116

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100       7.9G      1.463      1.314      1.117         90        224: 100%|██████████| 220/220 [00:24<00:00,  9.05it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 28/28 [00:02<00:00, 13.73it/s]


                   all        879       7139      0.327       0.26      0.227      0.114

100 epochs completed in 0.762 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 5.4MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 5.4MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics 8.3.80 🚀 Python-3.11.8 torch-2.4.1+cu121 CUDA:0 (NVIDIA H100 80GB HBM3, 81117MiB)
YOLO11n summary (fused): 100 layers, 2,584,882 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 28/28 [00:03<00:00,  8.10it/s]


                   all        879       7139      0.329      0.261       0.23      0.116
    Aortic enlargement        610       1446      0.591      0.492      0.562      0.327
           Atelectasis         38         51        0.3      0.252      0.163     0.0769
         Calcification         89        189          0          0      0.014    0.00541
          Cardiomegaly        455       1065      0.686      0.666      0.706      0.466
         Consolidation         61         85      0.361      0.212       0.21     0.0966
                   ILD         74        191      0.147       0.45        0.2     0.0853
          Infiltration        119        240      0.355      0.204      0.206     0.0913
          Lung Opacity        254        465      0.321      0.214      0.183     0.0702
           Nodule/Mass        157        418      0.452     0.0494      0.063     0.0314
          Other lesion        238        464      0.149      0.138     0.0675     0.0213
      Pleural effusio